# ***TPS-03,22***

## **Problem:**

For the March editionof the 2022 Tabular Playground Series you're challenged to forecast twelve-hours of traffic flow in a U.S. metropolis. The time series in this dataset are labelled with both location coordinates and a direction of travel -- a combination of features that will test your skill at spatio-temporal forecasting within a highly dynamic traffic network.

**Today I'm trying not changing baseline data's preprocessing, Only I'm apply Xgboost and StratifiedKFold**

# **Import Necessary Library**

In [ ]:
import subprocess
import pandas as pd
from pathlib import Path
from math import sin, cos, pi
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, roc_auc_score
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import enum
import math
#import shap
#shap.initjs()

import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor

def random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# **Load Data**

In [ ]:
data_path = Path('/kaggle/input/tabular-playground-series-mar-2022/')

train = pd.read_csv(data_path / 'train.csv', index_col='row_id', parse_dates=['time'])
test = pd.read_csv(data_path / 'test.csv', index_col='row_id', parse_dates=['time'])
submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='row_id')

In [ ]:
train.head()

# **Preprocessing of data's ----> Cat-Num**

In [ ]:
sin_vals = {
    'NB': 0.0,
    'NE': sin(1 * pi/4),
    'EB': 1.0,
    'SE': sin(3 * pi/4),
    'SB': 0.0,
    'SW': sin(5 * pi/4),    
    'WB': -1.0,    
    'NW': sin(7 * pi/4),  
}

cos_vals = {
    'NB': 1.0,
    'NE': cos(1 * pi/4),
    'EB': 0.0,
    'SE': cos(3 * pi/4),
    'SB': -1.0,
    'SW': cos(5 * pi/4),    
    'WB': 0.0,    
    'NW': cos(7 * pi/4),  
}


train['sin'] = train['direction'].map(sin_vals)
test['sin'] = test['direction'].map(sin_vals)

train['cos'] = train['direction'].map(cos_vals)
test['cos'] = test['direction'].map(cos_vals)

encoded_vals = {
    'NB': 0,
    'NE': 1,
    'EB': 2,
    'SE': 3,
    'SB': 4,
    'SW': 5,
    'WB': 6, 
    'NW': 7,
}

train['direction'] = train['direction'].map(encoded_vals)
test['direction'] = test['direction'].map(encoded_vals)


In [ ]:
train['year'] = train['time'].dt.year
train['month'] = train['time'].dt.month
train['day'] = train['time'].dt.day
train['hour'] = train['time'].dt.hour
train['minute'] = train['time'].dt.minute
train['weekday'] = train['time'].dt.weekday

test['year'] = test['time'].dt.year
test['month'] = test['time'].dt.month
test['day'] = test['time'].dt.day
test['hour'] = test['time'].dt.hour
test['minute'] = test['time'].dt.minute
test['weekday'] = test['time'].dt.weekday

train = train.drop('time', axis='columns')
test = test.drop('time', axis='columns')

## **After Preprocessing of Data**

In [ ]:
train.sample(3)

# **Build Model---> XGB + SKFold**

In [ ]:
#Feature selection and xgb parameters

columns = [col for col in train.columns if col not in ['row_id', 'congestion'] ]

xgb_params=  {'learning_rate': 0.0016596,
              'eval_metric': 'mae',
              'max_depth': 10,
              'alpha': 0.5108154566815425,
              'gamma': 1.9276236172849432,
              'reg_lambda': 11.40999855634382,
              'colsample_bytree': 0.705851334291963,
              'subsample': 0.8386116751473301,
              'min_child_weight': 2.5517043283716605,
}

xgb_params['objective'] = 'reg:tweedie'
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'
xgb_params['n_jobs'] = 4

In [ ]:
class Config(enum.Enum):
    SEED = 42
    N_FOLDS = 5
    EARLY_STOP = 100

In [ ]:
%%time

random_seed(Config.SEED.value)

mae_score = []

targets = train['congestion'].values

kf = StratifiedKFold(n_splits = Config.N_FOLDS.value, shuffle=True, random_state=Config.SEED.value)    
        
oof = np.zeros((train.shape[0],))
test_preds = 0

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(X=train, y=targets))):
        df_train, df_val = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
        train_target, val_target = targets[train_idx], targets[val_idx]
        
        train_df = xgb.DMatrix(df_train, label=train_target)
        val_df = xgb.DMatrix(df_val, label=val_target)
        test_df = xgb.DMatrix(test[columns])
        watchlist  = [ (train_df,'train'),(val_df,'eval')]
        model =  xgb.train(
            xgb_params,
            train_df, 
            30000, 
            watchlist, 
            early_stopping_rounds=Config.EARLY_STOP.value,
            verbose_eval=3000
        )
        
        oof_tmp = model.predict(val_df)
        test_tmp = model.predict(test_df)
        oof[val_idx] = oof_tmp
        test_preds += test_tmp/Config.N_FOLDS.value
        mae = mean_absolute_error(val_target, oof_tmp)
        mae_score.append(mae)
        print(f'FOLD: {f} MAE: {mae} Mean MAE: {np.mean(mae_score)}')

# **Prediction_Output**

In [ ]:
sub = pd.read_csv(data_path / 'sample_submission.csv')
sub['congestion'] = test_preds
sub.to_csv('submission1.csv', index=False)
sub.sample(3)

## **Thank-you for visiting guys**

Reference:
1. https://www.kaggle.com/santhoshkumarv/autism-xgboost-shap-values
2. https://www.kaggle.com/inversion/tps-mar-22-cyclical-features